# Dataest from kaggle

Dataset: https://www.kaggle.com/datasets/shiv28/next-word

# Import libraries

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding , LSTM , Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

# Load and Pre-process the data

In [2]:
# Load data file
file = open('D://Projects/Next Word Prediction/Data/next_word_prediction.txt','r',encoding='ISO-8859-1')

# Store file in list
lines = []
for i in file:
    lines.append(i)
    
# Convert list to string
data = ""
for i in lines:
    data = ' '.join(lines)
    
# Replace unnecessary stuff with space
lis = [',','\n','\r','\ufeff','_','-','.',';','!',':','*','?','â']
for i in lis:
    data = data.replace(i,'')
# Remove unnecessary space
data = data.split()
data = ' '.join(data)

In [3]:
data.split('3')[1]

' Not all that Mrs Bennet however with the assistance of her five daughters could ask on the subject was sufficient to draw from her husband any satisfactory description of Mr Bingley They attacked him in various ways with barefaced questions ingenious suppositions and distant surmises but he eluded the skill of them all and they were at last obliged to accept the secondhand intelligence of their neighbour Lady Lucas Her report was highly favourable Sir William had been delighted with him He was quite young wonderfully handsome extremely agreeable and to crown the whole he meant to be at the next assembly with a large party Nothing could be more delightful To be fond of dancing was a certain step towards falling in love and very lively hopes of Mr Bingleyâ\x80\x99s heart were entertained â\x80\x9cIf I can but see one of my daughters happily settled at Netherfieldâ\x80\x9d said Mrs Bennet to her husband â\x80\x9cand all the others equally well married I shall have nothing to wish forâ\x

In [4]:
# Split the data and take the first 2 chapters into x
text = data.split('3')[0]

In [5]:
# Check the len of our data
len(data)

908595

# Apply the Tokenization

In [6]:
# import tokenizer into variable and fit it
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

In [7]:
# Save it in a file
pickle.dump(tokenizer,open('token.pkl','wb'))

In [8]:
# convert the text
sequence_data = tokenizer.texts_to_sequences([text])[0]

In [9]:
sequence_data[:16]

[87, 209, 10, 12, 5, 128, 210, 211, 8, 5, 88, 59, 17, 129, 3, 5]

In [10]:
len(sequence_data)

1651

In [11]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

557


In [12]:
sequences = []
for i in range(3,len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
print(len(sequences))
sequences = np.array(sequences)
sequences[:10]

1648


array([[ 87, 209,  10,  12],
       [209,  10,  12,   5],
       [ 10,  12,   5, 128],
       [ 12,   5, 128, 210],
       [  5, 128, 210, 211],
       [128, 210, 211,   8],
       [210, 211,   8,   5],
       [211,   8,   5,  88],
       [  8,   5,  88,  59],
       [  5,  88,  59,  17]])

In [13]:
# Split the data into X , y
X = []
y = []

for i in sequences:
    X.append(i[:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [14]:
# Convert y to 2D array
y = to_categorical(y,num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Creating the model

In [15]:
model = Sequential()
model.add(Embedding(vocab_size,10,input_length=3))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             5570      
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 557)               557557    
                                                                 
Total params: 13,612,127
Trainable params: 13,612,127
Non-trainable params: 0
_________________________________________________________________


# Build our model

In [17]:
checkpoint = ModelCheckpoint('next_word.h5',monitor='loss',verbose=1,save_best_only=True)
model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.001))
model.fit(X,y,epochs=100,batch_size=64,callbacks=[checkpoint])

Epoch 1/100
26/26 [==============================] - ETA: 0s - loss: 6.0486
Epoch 1: loss improved from inf to 6.04863, saving model to next_word.h5
26/26 [==============================] - 27s 453ms/step - loss: 6.0486
Epoch 2/100
26/26 [==============================] - ETA: 0s - loss: 5.7037
Epoch 2: loss improved from 6.04863 to 5.70372, saving model to next_word.h5
26/26 [==============================] - 12s 466ms/step - loss: 5.7037
Epoch 3/100
26/26 [==============================] - ETA: 0s - loss: 5.6115
Epoch 3: loss improved from 5.70372 to 5.61146, saving model to next_word.h5
26/26 [==============================] - 13s 519ms/step - loss: 5.6115
Epoch 4/100
26/26 [==============================] - ETA: 0s - loss: 5.5739
Epoch 4: loss improved from 5.61146 to 5.57391, saving model to next_word.h5
26/26 [==============================] - 12s 478ms/step - loss: 5.5739
Epoch 5/100
26/26 [==============================] - ETA: 0s - loss: 5.5106
Epoch 5: loss improved from 5.57

26/26 [==============================] - 12s 475ms/step - loss: 2.4338
Epoch 38/100
26/26 [==============================] - ETA: 0s - loss: 2.3520
Epoch 38: loss improved from 2.43376 to 2.35201, saving model to next_word.h5
26/26 [==============================] - 13s 487ms/step - loss: 2.3520
Epoch 39/100
26/26 [==============================] - ETA: 0s - loss: 2.1929
Epoch 39: loss improved from 2.35201 to 2.19292, saving model to next_word.h5
26/26 [==============================] - 15s 565ms/step - loss: 2.1929
Epoch 40/100
26/26 [==============================] - ETA: 0s - loss: 1.9976
Epoch 40: loss improved from 2.19292 to 1.99763, saving model to next_word.h5
26/26 [==============================] - 14s 530ms/step - loss: 1.9976
Epoch 41/100
26/26 [==============================] - ETA: 0s - loss: 1.8787
Epoch 41: loss improved from 1.99763 to 1.87872, saving model to next_word.h5
26/26 [==============================] - 14s 530ms/step - loss: 1.8787
Epoch 42/100
26/26 [=====

Epoch 75/100
26/26 [==============================] - ETA: 0s - loss: 0.3117
Epoch 75: loss did not improve from 0.20832
26/26 [==============================] - 13s 501ms/step - loss: 0.3117
Epoch 76/100
26/26 [==============================] - ETA: 0s - loss: 0.2492
Epoch 76: loss did not improve from 0.20832
26/26 [==============================] - 13s 491ms/step - loss: 0.2492
Epoch 77/100
26/26 [==============================] - ETA: 0s - loss: 0.1940
Epoch 77: loss improved from 0.20832 to 0.19399, saving model to next_word.h5
26/26 [==============================] - 12s 456ms/step - loss: 0.1940
Epoch 78/100
26/26 [==============================] - ETA: 0s - loss: 0.1778
Epoch 78: loss improved from 0.19399 to 0.17779, saving model to next_word.h5
26/26 [==============================] - 12s 448ms/step - loss: 0.1778
Epoch 79/100
26/26 [==============================] - ETA: 0s - loss: 0.1687
Epoch 79: loss improved from 0.17779 to 0.16868, saving model to next_word.h5
26/26 [==

# Prediction

In [18]:
# Load the model and tokenizer
model = load_model('next_word.h5')
tokenizer = pickle.load(open('token.pkl','rb'))

In [19]:
def predict_next_word(model,tokenizer,text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds = np.argmax(model.predict(sequence))
    predicted_word = ""
    
    for key , value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    print(predicted_word)
    return predicted_word

In [27]:
while True:
    text = input('Enter your line')
    if text == '0':
        print('Execution completed')
        break
    else:
        try:
            text = text.split(" ")
            text = text[-3:]
            print(text)
            predict_next_word(model,tokenizer,text)
        except Exception as e:
            print('error eccurred: ',e)
            continue

Enter your lineimaginary Vanity and
['imaginary', 'Vanity', 'and']
1/1 [==============================] - 0s 45ms/step
did
Enter your linewere married to
['were', 'married', 'to']
1/1 [==============================] - 0s 45ms/step
an
Enter your linecharming amusement for
['charming', 'amusement', 'for']
1/1 [==============================] - 0s 44ms/step
i
Enter your line0
Execution completed
